In [1]:
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from annual_profit_optimisation import run_profit_optimisation

# Set ERCOT load zone and solar capacity
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
SOLAR_CAPACITY = 300
SOLAR_PPA_DOL_KWH = 250
FACILITY_LIFETIME = 25

In [2]:
# READ IN DATA
# TODO: update as 'electricity' here is a normalised 8760 solar profile
hourly_dayahead_realtime_solar = pd.read_csv("hourly_dayahead_realtime_solar.csv")

# Filter by load zone and complete data
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar[hourly_dayahead_realtime_solar['zone'] == ERCOT_LOAD_ZONE]
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz[hourly_dayahead_realtime_solar_lz['electricity'].notna()]

number_of_days_test = 1
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz.iloc[0:24*number_of_days_test,]
# Sets time duration of the optimisation
# TODO: generat appropriate 8760 - due to shift in UTC time used for solar profile 
# and local_time used for ERCOT realtime and dayahead prices
T = hourly_dayahead_realtime_solar_lz.shape[0]
print(T)

24


In [3]:
# FACILITY
FACILITY_LIFETIME = 25
OFFTAKE_TPD = 10
GREEN_THRESHOLD_kg_per_kg = 0.4
NUM_DAYS = math.ceil(T/24)

# POWER
#   SOLAR
SOLAR_CAPACITY = 400
SOLAR_PPA_DOL_KWH = 0.03 # 2 cents per kWh ($USD 50 per MWh)
hourly_solar_production_kwh = hourly_dayahead_realtime_solar_lz['electricity']/hourly_dayahead_realtime_solar_lz['electricity'].max()
#   GRID 
#       CONVERT $/MWh -> $/kWh
hourly_dayahead_dol_per_kwh = hourly_dayahead_realtime_solar_lz['price_dayahead'] / 1000
ERCTO_CO2_kg_per_kwh = 0.42062


# EQUIPMENT
#   OPERATIONAL 
#       EFFICIENCY
ELECTROLYSIS_EFFICIENCY = 55
COMPRESSION_EFFICIENCY = 4
LIQUEFACTION_EFFICIENCY = 10
#       LOWER OPERATION BOUNDS (% of nameplate capacity)
ELECTROLYSIS_OPERATION_LOWER = 0.15
COMPRESSOR_OPERATION_LOWER = 0.15
LIQUEFACTION_OPERATION_LOWER = 0.60 
#       RAMP CONSTRAINTS (% of nameplate capacity)
LIQUEFACTION_MAX_UP = 0.2 # 10% per hour up
LIQUEFACTION_MAX_DOWN = 0.2 # 10% per hour down

#   CAPEX
#   CAPEX - total
ELECTROLYSIS_CAPEX_DOL_TPD = 1900000 # 1.9 million $USD per TPD of electrolysis
COMPRESSOR_CAPEX_DOL_TPD = 900000 # 0.8 million $USD per TPD of comrpessor
LIQUEFACTION_CAPEX_DOL_TPD = 2945000 # 2.945 million $USD per TPD of liquefaction per day
LIQUEFACTION_CAPEX_DOL_TPD = 2945000 # 2.945 million $USD per TPD of liquefaction per day

#   CAPEX - daily, for fast testing 
ELECTROLYSIS_CAPEX_DOL_kgPD = ELECTROLYSIS_CAPEX_DOL_TPD / 1000
COMPRESSOR_CAPEX_DOL_kgPD = COMPRESSOR_CAPEX_DOL_TPD / 1000
LIQUEFACTION_CAPEX_DOL_kgPD = LIQUEFACTION_CAPEX_DOL_TPD / 1000


ELECTROLYSIS_CAPEX_DOL_kgPD = ELECTROLYSIS_CAPEX_DOL_kgPD * (NUM_DAYS / (FACILITY_LIFETIME * 365) )
COMPRESSOR_CAPEX_DOL_kgPD = COMPRESSOR_CAPEX_DOL_kgPD * (NUM_DAYS / (FACILITY_LIFETIME * 365) )
LIQUEFACTION_CAPEX_DOL_kgPD = LIQUEFACTION_CAPEX_DOL_kgPD * (NUM_DAYS / (FACILITY_LIFETIME * 365) )

In [5]:
problem, operations, facility = run_profit_optimisation(
        SOLAR_CAPACITY,
        SOLAR_PPA_DOL_KWH,
        hourly_solar_production_kwh,
        hourly_dayahead_dol_per_kwh,
        ELECTROLYSIS_EFFICIENCY,
        COMPRESSION_EFFICIENCY,
        LIQUEFACTION_EFFICIENCY,
        LIQUEFACTION_MAX_UP,
        LIQUEFACTION_MAX_DOWN,
        15,
        NUM_DAYS,
        ELECTROLYSIS_CAPEX_DOL_kgPD,
        COMPRESSOR_CAPEX_DOL_kgPD,
        LIQUEFACTION_CAPEX_DOL_kgPD,
        GREEN_THRESHOLD_kg_per_kg,
        ERCTO_CO2_kg_per_kwh,
        T
        )
# OUTPUT
power_opex_total = np.sum(operations['hourly_dayahead_dol_per_kwh'] @ operations['realtime_production_kwh']) 
power_opex_total += np.sum(SOLAR_PPA_DOL_KWH * operations['solar_production_kwh'])
power_opex_per_kg = power_opex_total/sum(operations['liquefaction_produced_kg'])

capex_total = facility['electrolyser_capacity_ph'] * 24 *ELECTROLYSIS_CAPEX_DOL_kgPD + \
    facility['compressor_capacity_ph'] * 24 * COMPRESSOR_CAPEX_DOL_kgPD + \
   facility['liquefaction_capacity_ph'] * 24 * LIQUEFACTION_CAPEX_DOL_kgPD


capex_per_kg = capex_total / sum(operations['liquefaction_produced_kg'])
print(f"total offtake {sum(operations['liquefaction_produced_kg'])}")
print(f"power opex / kg {power_opex_per_kg}")
print(f"capex per kg {capex_per_kg}")
print(f"total dol per kg {power_opex_per_kg + capex_per_kg}")

print(f"capacity of electrolysis (tpd): {facility['electrolyser_capacity_ph']*24}, capex {facility['electrolyser_capacity_ph'] *24 *ELECTROLYSIS_CAPEX_DOL_kgPD}")
print(f"capacity of compressor (tpd) {facility['compressor_capacity_ph']*24}, capex {facility['compressor_capacity_ph'] * 24 * COMPRESSOR_CAPEX_DOL_kgPD}")
print(f"capacity of liquefaction (tpd) {facility['liquefaction_capacity_ph']*24}, capex {facility['liquefaction_capacity_ph']* 24 * LIQUEFACTION_CAPEX_DOL_kgPD}")

total offtake 14.999999999999996
power opex / kg 2.071603404707625
capex per kg 2.0627010110933544
total dol per kg 4.13430441580098
capacity of electrolysis (tpd): 49.04942574677225, capex 10.213031114396413
capacity of compressor (tpd) 49.04942574677225, capex 4.837751580503564
capacity of liquefaction (tpd) 49.23389093461477, capex 15.889732471500329


In [ ]:
# Define decision variables
#   Electricity decision variables


# OUTPUT
power_opex_total = np.sum(hourly_dayahead_dol_per_kwh @ realtime_consumption.value) 
power_opex_total += np.sum(SOLAR_PPA_DOL_KWH * solar_consumed_facility.value)
power_opex_per_kg = power_opex_total/sum(liquefacion_throughput.value)

capex_total = electrolyser_nameplate_capacity_hour.value * 24 *ELECTROLYSIS_CAPEX_DOL_kgPD + \
    compressor_nameplate_capacity_hour.value * 24 * COMPRESSOR_CAPEX_DOL_kgPD + \
    liquefaction_nameplate_capacity_hour.value * 24 * LIQUEFACTION_CAPEX_DOL_kgPD


capex_per_kg = capex_total / sum(liquefacion_throughput.value)
print(f"total offtake {sum(liquefacion_throughput.value)}")
print(f"power opex / kg {power_opex_per_kg}")
print(f"capex per kg {capex_per_kg}")
print(f"total dol per kg {power_opex_per_kg + capex_per_kg}")

print(f"capacity of electrolysis (tpd): {electrolyser_nameplate_capacity_hour.value*24}, capex {electrolyser_nameplate_capacity_hour.value *24 *ELECTROLYSIS_CAPEX_DOL_kgPD}")
print(f"capacity of compressor (tpd) {compressor_nameplate_capacity_hour.value*24}, capex {compressor_nameplate_capacity_hour.value * 24 * COMPRESSOR_CAPEX_DOL_kgPD}")
print(f"capacity of liquefaction (tpd) {liquefaction_nameplate_capacity_hour.value*24}, capex {liquefaction_nameplate_capacity_hour.value * 24 * LIQUEFACTION_CAPEX_DOL_kgPD}")

In [ ]:
print(ELECTROLYSIS_CAPEX_DOL_kgPD)

In [ ]:
print(LIQUEFACTION_CAPEX_DOL_kgPD)

In [ ]:

# create dataframe of results: 
operations_df = pd.DataFrame({
    "realtime_price_$_per_kwh": hourly_dayahead_dol_per_kwh,
    "realtime_production_kwh":realtime_consumption.value,
    "solar_production_kwh": solar_consumed_facility.value, 
    "electrolyser_consumption_kwh": ELECTROLYSIS_EFFICIENCY*electrolyser_throughput.value, 
    "compressor_consumption_kwh": COMPRESSION_EFFICIENCY*compressor_throughput.value, 
    "liquefaction_consumption_kwh": LIQUEFACTION_EFFICIENCY*liquefacion_throughput.value,
    "electrolyser_produced_kg": electrolyser_throughput.value, 
    "compressor_produced_kg": compressor_throughput.value, 
    "liquefaction_produced_kg": liquefacion_throughput.value,
    "compress_to_liquefaction": compressor_to_liquefaction.value,
    "gh2_storage_level_kg": gh2_storage_level.value[0:T],
    # TODO: if problem size allows, add in big-M constraint 
    "gh2_storage_net_inflow": gh2_storage_inflow.value[0:T] - gh2_storage_outflow.value[0:T],
    "gh2_storge_inflow_kg": gh2_storage_inflow.value[0:T], 
    "gh2_storage_outflow_kg": gh2_storage_outflow.value[0:T]
    # "gh2_storage_netflow": gh2_storage_netflow.value
})
operations_df.to_csv("operations_8760_df.csv")

In [ ]:
fig, axs = plt.subplots(7, 1, figsize=(16, 16), sharex=True)  # 'sharex=True' shares the x-axis among all plots

t = range(24)
# Plot each time series in a different subplot
axs[0].plot(t, electrolyser_throughput.value, label='Electrolyser throughput (kg)')
axs[0].set_title('Electrolyser throughput (kg)')
axs[0].legend()

axs[1].plot(t, compressor_throughput.value, label='compressor_throughput (kg)')
axs[1].set_title('Compressor_throughput.value')
axs[1].legend()

axs[2].plot(t, liquefacion_throughput.value, label='Liqufeaction throughput (kg)')
axs[2].set_title('Liqufeaction throughput (kg)')
axs[2].legend()

axs[3].plot(t, gh2_storage_level.value[0:T], label='GH2 storage level (kg)')
axs[3].set_title('GH2 storage level (kg)')
axs[3].legend()

axs[4].plot(t, operations_df['gh2_storage_net_inflow'].to_list(), label='GH2 storage net inflow (kg)')
axs[4].set_title('GH2 net inflow (kg)')
axs[4].legend()

axs[5].plot(t, operations_df['solar_production_kwh'].to_list(), label='Solar consumed (kWh)')
axs[5].set_title('Solar consumed (kWh)')
axs[5].legend()



axs6 = axs[6]
axs6.plot(t, operations_df['realtime_production_kwh'].to_list(), label='Grid consumed (kWh)', color='blue')
axs6.set_title('Grid consumed (kWh)')
axs6.set_ylabel('Grid consumed (kWh)', color='blue')
axs6.legend()

# Create a secondary y-axis for the fourth subplot
axs3_right = axs6.twinx()
axs3_right.plot(t, operations_df['realtime_price_$_per_kwh'].to_list(), label='Real time price', color='green')
axs3_right.set_ylabel('Realtime price', color='green')
axs3_right.legend()
# Optionally, adjust spacing between subplots for better visibility
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
operations_df['gh2_storage_net_inflow']